# Quantitative Risk Management

Click <a href="https://colab.research.google.com/github/Lolillosky/QuantRiskManagement/blob/main/NOTEBOOKS/8_IRS_Sensitivities.ipynb">
    <img src="https://upload.wikimedia.org/wikipedia/commons/d/d0/Google_Colaboratory_SVG_Logo.svg" width="30" alt="Google Colab">
</a> to open this notebook in Google Colab.


In this exercise, you are going to bootstrap a zero coupon curve from par swap rates. With this curve and a given volatility, you are going to price a swaption using the Bachelier formula. Then, you are going to compute, with the help of AD, the sensitivities of the swaption with respect to zero coupon rates. Then, you will transform these to sensitivities to par swap rates.

## Import main libraries:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import pandas as pd
from scipy.optimize import fsolve


## Import the following libraries from the repository:

In [ ]:
import sys
sys.path.append('../CODE')  # Adjust the path as necessary

In [ ]:
import pytorch_spline
import pytorch_option_formulas
import pytorch_ir_curve

## Load the swap data

In '../DATA/IR_Swap.csv', you have par swap rates for different maturities:

In [ ]:
swap_rates = pd.read_csv('../DATA/IR_Swap.csv')

swap_rates['Rate'] /= 100

plt.plot(swap_rates['Mat (yrs)'], swap_rates['Rate'], '.')
plt.ylim((0))

## Explore Pytorch spline function

In pytorch_spline.py module, you have a Pythorch implementation of a Pytorch spline function. Let us see how it works:

In [ ]:
# We generate a discrete representation of the sin curve with numpy.
x = np.linspace(0,2*np.pi, 6)
y = np.sin(x)

# We generate Pytorch tensors from the numpy arrays.
x = torch.tensor(x, requires_grad=False) # We do not want to compute derivatives with respect to these.
y = torch.tensor(y, requires_grad=True) # We want to compute derivatives with respect to these.

# We generate a populated Pytorch tensor in the x dimension.
x_grid = torch.linspace(0,2*np.pi, 100,requires_grad=False) # We do not want to compute derivatives with respect to these.

# To initialize the spline, we just have to instanciate it with the data points.
spline = pytorch_spline.NaturalCubicSpline_Torch(x,y)

# We compute the spline interpolation for the populated grid.
with torch.no_grad():
    y_grid = spline.evaluate_spline(x_grid)

# We plot the spline interpolation for the populated grid together with the discrete data points used to build the spline.
plt.plot(x, y.detach(), 'o')
plt.plot(x_grid, y_grid)

# For a given x we compute the y interpolated by the spline function.
x_0 = torch.tensor(5.3, requires_grad=False)
y_0 = spline.evaluate_spline(x_0)

# We compute the derivative of y_0 with respect to the spline inputs.
y_0.backward()

grad = y.grad

# We plot the gradients.

plt.bar(x = x, height= grad, width= 0.1, color = 'green')
plt.plot(x_0, y_0.detach(), 'x')
plt.axhline(linestyle = '-.')



In [ ]:
time = torch.tensor(swap_rates['Mat (yrs)'].values, requires_grad=False)
rates = torch.tensor(swap_rates['Rate'].values, requires_grad=False) 

curve = pytorch_ir_curve.IR_Curve(time, rates)

time_grid = torch.linspace(0,50,100, requires_grad=False)

zc_rates = curve.zero_coupon_rates(time_grid)

plt.plot(time, rates, 'o')
plt.plot(time_grid, zc_rates)
plt.ylim(0.0)




In [ ]:
swap = pytorch_ir_curve.IR_Swap(0,5,1)

swap.calc_par_rate(curve).item()

In [ ]:
fitter = CurveFitter(swap_rates['Mat (yrs)'].values, swap_rates['Rate'].values, 1.0)

zc_rates = fitter.fit()

curve = pytorch_ir_curve.IR_Curve(torch.tensor(swap_rates['Mat (yrs)'].values), torch.tensor(zc_rates))

In [ ]:
time_grid = torch.linspace(0,50,100, requires_grad=False)

zc_rates_grid = curve.zero_coupon_rates(time_grid)

plt.plot(swap_rates['Mat (yrs)'], swap_rates['Rate'], 'o')
plt.plot(time_grid, zc_rates)


In [ ]:
@write the bachelier formula in pytorch
